In [7]:
from svgpathtools import parse_path, Line, Path, wsvg
from svgpathtools import Path, Line, QuadraticBezier, CubicBezier, Arc

def offset_curve(path, offset_distance, steps=1000):
    """Takes in a Path object, `path`, and a distance,
    `offset_distance`, and outputs an piecewise-linear approximation 
    of the 'parallel' offset curve."""
    nls = []
    for seg in path:
        for k in range(steps):
            t = k / steps
            offset_vector = offset_distance * seg.normal(t)
            nl = Line(seg.point(t), seg.point(t) + offset_vector)
            nls.append(nl)
    connect_the_dots = [Line(nls[k].end, nls[k+1].end) for k in range(len(nls)-1)]
    if path.isclosed():
        connect_the_dots.append(Line(nls[-1].end, nls[0].end))
    offset_path = Path(*connect_the_dots)
    return offset_path

# Examples:
path1 = Path(CubicBezier(300+100j, 100+100j, 200+200j, 200+300j))  # A cubic beginning at (300, 100) and ending at (200, 300)
path2 = Line(200+300j, 250+350j)  # A line beginning at (200, 300) and ending at (250, 350)
paths = [path1, path2]

offset_distances = [10*k for k in range(1,51)]
offset_paths = []
for path in paths:
    for distances in offset_distances:
        offset_paths.append(offset_curve(path, distances))

# Let's take a look
wsvg(paths + offset_paths, 'g'*len(paths) + 'r'*len(offset_paths), filename='offset_curves.svg')

AttributeError: 'complex' object has no attribute 'normal'

In [3]:
!code offset_curves.svg